# 126 Project Traffic Simulator

Let the road be length $n$. Cars start at position 0 and reach their destination at position $n-1$. Each car has an exponential "clock" with parameter $\gamma$, and every time their clock goes off they move forward with probability $p$. If a car is directly in front of them, they do not move regardless. A car enters the road at position 0 at a rate of $Exp(\alpha)$ and exit the road at position $n-1$ with a rate of $Exp(\beta)$.

### Choose global variables:

In [19]:
from ipywidgets import interactive
from IPython.display import display
from IPython.display import clear_output

def set_params(n, alpha, beta, theta, gamma, p, seed):
    return [n, alpha, beta, theta, gamma, p, seed]

params = interactive(set_params, n=(1, 20), alpha=(0.0, 2.0), beta=(0.0, 2.0), theta=(0.0, 0.5), gamma=(0.0, 2.0), p=(0.0, 1.0), seed=(1, 999))
display(params)

interactive(children=(IntSlider(value=10, description='n', max=20, min=1), FloatSlider(value=1.0, description=…

### Set global variables:
(Note: will break if you set alpha = 0 due to 0 division error)
Remember to rerun this cell every time a slider is updated

In [65]:
n, alpha, beta, theta, gamma, p, seed = params.result

### Define particle and environment classes:

In [58]:
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output


class Particle:
    
    count = 0
    
    def __init__(self, gamma=gamma, p=p):
        Particle.count += 1
        self.id = Particle.count - 1
        self.gamma = gamma
        self.p = p
        self._position = 0
        
    def update_position(self, value=1):
        self._position += value
        
    def get_position(self):
        return self._position
        
class Environment:
    def __init__(self, n=n, alpha=alpha, beta=beta):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.particles = []
        self.occupied_positions = [0 for _ in range(n)]
        
    def run_simulation(self, time_len=60):
        Particle.count = 0 # slight issue where running repeated simulations wouldn't cause particle counts to reset
        
        sys.stdout.write("\r{}".format(self.occupied_positions))
        sys.stdout.flush()
        
        curr_time = 0
        time_lapse = np.random.exponential(scale=1/self.alpha)
        time.sleep(time_lapse)
        curr_time += time_lapse
        particle = Particle(gamma=gamma, p=p)
        self.particles.append(particle)
        self.occupied_positions[0] = 1
        
        actions = [(curr_time, particle.id, particle.get_position())]
        entrances = 1
        exits = 0
        
        travel_times = [[0, None]]
        
        while curr_time < time_len:
            sys.stdout.write("\r{}".format(self.occupied_positions))
            sys.stdout.flush()
            
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + self.alpha + self.beta
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha, self.beta]) / merged_param
            particle = np.random.choice(self.particles + ['entrance', 'exit'], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                if self.occupied_positions[0]:
                    continue
                else:
                    new_particle = Particle(gamma=gamma, p=p)
                    self.particles.append(new_particle)
                    self.occupied_positions[0] = 1
                    
                    actions += [(curr_time, new_particle.id, new_particle.get_position())]
                    entrances += 1
                    
                    travel_times.append([curr_time, None])
                    
            elif particle == 'exit':
                if self.occupied_positions[self.n-1]: # changed to refer to self.n instead of n
                    end_particle = self.particles.pop(0)
                    self.occupied_positions[self.n-1] = 0 # changed to refer to self.n instead of n
                    
                    actions += [(curr_time, end_particle.id, 'exit')]
                    exits += 1
                    
                    travel_times[end_particle.id][1] = curr_time
                    
                else:
                    continue
            else:
                particle_pos = particle.get_position()
                if particle_pos == self.n - 1 or self.occupied_positions[particle_pos + 1] == 1: # changed to refer to self.n instead of n
                    continue
                else:
                    self.occupied_positions[particle_pos] = 0
                    self.occupied_positions[particle_pos + 1] = 1
                    particle.update_position()
                    
                    actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        travel_times1 = [end - begin for begin, end in travel_times if end]
        if len(travel_times1) > 0:
            print("Travel times: {}".format([round(t, 2) for t in travel_times1]))
            print("Begin and End times: {}".format([(round(begin, 2), round(end, 2)) for begin, end in travel_times if end]))
            print('Average travel time: {}'.format(sum(travel_times1) / len(travel_times1)))
        else:
            print('No cars reached the end')
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Add 2D Environment:

In [67]:
class Particle2D(Particle):
    def __init__(self, gamma=gamma, p=p, direction=0): # 0 is horizontal, 1 is vertical
        Particle.__init__(self, gamma, p)
        self._position = [-1, -1]
        self._direction = direction
        
    def update_position(self, value=1):
        self._position[self._direction] += value
        
    def set_position(self, value=[-1, -1]):
        self._position = value
        
    def get_direction(self):
        return self._direction
        
# FIRST INDEX COLUMN, SECOND INDEX ROAD
class Env2D(Environment):
    def __init__(self, num_roads=(1,1), n=n, alpha=alpha, beta=beta, theta=theta):
        Environment.__init__(self, n, alpha, beta)
        self.theta = theta
        self.occupied_positions = np.array([[0 for _ in range(n)] for _ in range(n)])
        self.horizontal_roads = np.random.choice(range(n), num_roads[0], replace=False)
        self.vertical_roads = np.random.choice(range(n), num_roads[1], replace=False)
        self.num_roads = num_roads
        self.intersections = []
        [[self.intersections.append((x, y)) for x in self.vertical_roads] for y in self.horizontal_roads]
        self.inter_directions = {}
        for i in self.intersections:
            self.inter_directions[i] = 0
        
    def run_simulation(self, time_len=60):
        Particle.count = 0 # slight issue where running repeated simulations wouldn't cause particle counts to reset
        
        print(self.occupied_positions)
        
        curr_time = 0
        time_lapse = np.random.exponential(scale=1/(self.alpha * sum(self.num_roads)))
        time.sleep(time_lapse)
        curr_time += time_lapse
        particle = Particle2D(gamma=gamma, p=p, direction=np.random.choice([0, 1], p=[self.num_roads[0]/sum(self.num_roads), self.num_roads[1]/sum(self.num_roads)]))
        self.particles.append(particle)
        if not particle.get_direction():
            road = np.random.choice(self.horizontal_roads)
            particle.set_position([0, road])
            self.occupied_positions[0, road] = 1
        else:
            road = np.random.choice(self.vertical_roads)
            particle.set_position([road, 0])
            self.occupied_positions[road, 0] = 1
        
        actions = [(curr_time, particle.id, particle.get_position())]
        entrances = 1
        exits = 0
        
        while curr_time < time_len:
            clear_output(wait=True)
            print(self.occupied_positions)
            
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + (self.alpha + self.beta) * sum(self.num_roads) + self.theta * (sum(self.num_roads) ** 2)  
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha for _ in range(sum(self.num_roads))] + [self.beta for _ in range(sum(self.num_roads))] + [self.theta for _ in range(sum(self.num_roads) ** 2)]) / merged_param
            particle = np.random.choice(self.particles + ['entrance' for _ in range(sum(self.num_roads))] + ['exit' for _ in range(sum(self.num_roads))] + ['intersection' for _ in range(sum(self.num_roads) ** 2)], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                direction = np.random.choice([0, 1], p=[len(self.horizontal_roads)/sum(self.num_roads), len(self.vertical_roads)/sum(self.num_roads)])
                if not direction:
                    road = np.random.choice(self.horizontal_roads)
                    if self.occupied_positions[0][road]:
                        continue
                    new_particle = Particle2D(gamma=gamma, p=p, direction=direction)
                    self.particles.append(new_particle)
                    self.occupied_positions[0][road] = 1
                    new_particle.set_position([0, road])
                else:
                    road = np.random.choice(self.vertical_roads)
                    if self.occupied_positions[road][0]:
                        continue
                    new_particle = Particle2D(gamma=gamma, p=p, direction=direction)
                    self.particles.append(new_particle)
                    self.occupied_positions[road][0] = 1
                    new_particle.set_position([road, 0])
                
                    
                actions += [(curr_time, new_particle.id, new_particle.get_position())]
                entrances += 1
                    
            elif particle == 'exit':
                direction = np.random.choice([0, 1], p=[len(self.horizontal_roads)/sum(self.num_roads), len(self.vertical_roads)/sum(self.num_roads)])
                if not direction:
                    road = np.random.choice(self.horizontal_roads)
                    if self.occupied_positions[self.n-1][road]: # changed to refer to self.n instead of n
                        particle = [p for p in self.particles if p.get_position() == [self.n-1, road]][0]
                        self.particles.remove(particle)
                        self.occupied_positions[self.n-1][road] = 0 # changed to refer to self.n instead of n
                        actions += [(curr_time, particle.id, [n, road])]
                        exits += 1
                else:
                    road = np.random.choice(self.vertical_roads)
                    if self.occupied_positions[road][self.n-1]: # changed to refer to self.n instead of n
                        particle = [p for p in self.particles if p.get_position() == [road, self.n-1]][0]
                        self.particles.remove(particle)
                        self.occupied_positions[road][self.n-1] = 0 # changed to refer to self.n instead of n
                        actions += [(curr_time, particle.id, [road, n])]
                        exits += 1
            elif particle == 'intersection':
                i = np.random.choice(range(len(self.intersections)))
                intersection = self.intersections[i]
                self.inter_directions[intersection] = 0 if self.inter_directions[intersection] else 1
                actions += [(curr_time, 'intersection' + str(intersection), self.inter_directions[intersection])] 
            else:
                particle_pos = particle.get_position()
                direction = particle.get_direction()
                if not direction:
                    pos = (particle_pos[0] + 1, particle_pos[1])
                    if pos in self.intersections and self.inter_directions[pos]:
                        continue
                    elif particle_pos[0] == self.n - 1 or self.occupied_positions[particle_pos[0] + 1, particle_pos[1]] == 1:
                        continue
                    else:
                        self.occupied_positions[particle_pos[0]][particle_pos[1]] = 0
                        self.occupied_positions[particle_pos[0] + 1][particle_pos[1]] = 1
                        particle.update_position()
                        actions += [(curr_time, particle.id, particle.get_position())]
                else:
                    pos = (particle_pos[0], particle_pos[1] + 1)
                    if pos in self.intersections and not self.inter_directions[pos]:
                        continue
                    elif particle_pos[1] == self.n - 1 or self.occupied_positions[particle_pos[0], particle_pos[1] + 1] == 1:
                        continue
                    else:
                        self.occupied_positions[particle_pos[0]][particle_pos[1]] = 0
                        self.occupied_positions[particle_pos[0]][particle_pos[1] + 1] = 1
                        particle.update_position()
                        actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Run simulator:

In [ ]:
env = Env2D(num_roads=(3,3), n=n, alpha=alpha, b)
env.run_simulation()

[[0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 1 0 1 1]
 [1 1 1 1 1 1 1 1 0 0]
 [1 1 1 0 1 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 1 1 0 0 1 0 0 0 0]]


In [66]:
env1 = Environment(n=n, alpha=alpha, beta=beta)
env1.run_simulation(time_len = 10)

[0, 0, 1, 0, 0, 0, 1, 1, 1, 0]
 done
Summary:
Total entrances: 5
Total exits: 1
Travel times: [5.94]
Begin and End times: [(0, 5.94)]
Average travel time: 5.938412845689733

 Log:
Time: 1.0600787889770467
Particle: 0
New position: 0


Time: 1.2642519476228609
Particle: 0
New position: 1


Time: 1.3959670494852447
Particle: 0
New position: 2


Time: 1.783855009399567
Particle: 0
New position: 3


Time: 2.323977657377165
Particle: 0
New position: 4


Time: 2.4412498287392186
Particle: 1
New position: 0


Time: 3.2686412790740094
Particle: 1
New position: 1


Time: 3.291526122485012
Particle: 2
New position: 0


Time: 3.399995569359751
Particle: 0
New position: 5


Time: 3.4181246895037662
Particle: 1
New position: 2


Time: 3.4242455296020213
Particle: 0
New position: 6


Time: 3.541893655079793
Particle: 1
New position: 3


Time: 3.6750042743041402
Particle: 2
New position: 1


Time: 3.758839909829209
Particle: 0
New position: 7


Time: 4.01648210614889
Particle: 2
New position: 2


Tim